<a href="https://colab.research.google.com/github/elangbijak4/Riset-Smart-City/blob/main/Demo_AI_Driven_(Kasus_LSTM)_untuk_EMS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# 1. Simulasi data kendaraan per detik
def simulate_timeseries(n_seconds=3000):
    data = []
    for t in range(n_seconds):
        speed = np.random.randint(0, 100)
        slope = np.random.choice(['flat', 'uphill', 'downhill'])
        battery = np.round(np.random.uniform(0.0, 1.0), 2)

        # Aturan aksi
        if slope == 'uphill' and battery < 0.3:
            action = 'engine'
        elif speed < 40 and battery > 0.5:
            action = 'battery'
        else:
            action = np.random.choice(['battery', 'engine'])

        data.append([speed, slope, battery, action])
    return pd.DataFrame(data, columns=['speed', 'slope', 'battery', 'action'])

df = simulate_timeseries()

# 2. One-hot encoding untuk slope & label encode action
df = pd.get_dummies(df, columns=['slope'])  # slope_flat, slope_uphill, slope_downhill
label_enc = LabelEncoder()
df['action'] = label_enc.fit_transform(df['action'])  # battery=0, engine=1

# 3. Buat jendela waktu (10 detik)
window_size = 10
features = ['speed', 'battery', 'slope_downhill', 'slope_flat', 'slope_uphill']
X, y = [], []

for i in range(len(df) - window_size):
    X.append(df[features].iloc[i:i+window_size].values)
    y.append(df['action'].iloc[i + window_size])  # prediksi aksi pada waktu ke-(t+10)

X = np.array(X)
y = tf.keras.utils.to_categorical(np.array(y), num_classes=2)

# 4. Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 5. Model LSTM
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(64, return_sequences=True, input_shape=(window_size, X.shape[2])),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(2, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# 6. Training
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

# 7. Evaluasi
loss, acc = model.evaluate(X_test, y_test)
print(f"\n🎯 Akurasi LSTM di data uji: {acc:.2f}")

# 8. Fungsi prediksi dengan input urutan
def predict_action_sequence(sequence_df):
    window = sequence_df[features].tail(window_size).values.reshape(1, window_size, len(features))
    pred = model.predict(window)
    class_idx = np.argmax(pred)
    return label_enc.inverse_transform([class_idx])[0]

# 9. Contoh prediksi
print("\n🔍 Prediksi aksi berdasarkan 10 detik terakhir:")
print(predict_action_sequence(df.tail(10)))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 10, 64)         │        17,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 2)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 30,402 (118.76 KB)

 Trainable params: 30,402 (118.76 KB)

 Non-trainable params: 0 (0.00 B)

ValueError: Invalid dtype: object

In [9]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# 1. Simulasi data kendaraan per detik
def simulate_timeseries(n_seconds=3000):
    data = []
    for t in range(n_seconds):
        speed = np.random.randint(0, 100)
        slope = np.random.choice(['flat', 'uphill', 'downhill'])
        battery = np.round(np.random.uniform(0.0, 1.0), 2)

        # Aturan aksi
        if slope == 'uphill' and battery < 0.3:
            action = 'engine'
        elif speed < 40 and battery > 0.5:
            action = 'battery'
        else:
            action = np.random.choice(['battery', 'engine'])

        data.append([speed, slope, battery, action])
    return pd.DataFrame(data, columns=['speed', 'slope', 'battery', 'action'])

df = simulate_timeseries()

# 2. One-hot encoding untuk slope & label encode action
df = pd.get_dummies(df, columns=['slope'])  # slope_flat, slope_uphill, slope_downhill
label_enc = LabelEncoder()
df['action'] = label_enc.fit_transform(df['action'])  # battery=0, engine=1

# 3. Buat jendela waktu (10 detik)
window_size = 10
features = ['speed', 'battery', 'slope_downhill', 'slope_flat', 'slope_uphill']
X, y = [], []

for i in range(len(df) - window_size):
    X.append(df[features].iloc[i:i+window_size].values)
    y.append(df['action'].iloc[i + window_size])  # prediksi aksi pada waktu ke-(t+10)

X = np.array(X).astype(np.float32) # Ensure X is float32
y = tf.keras.utils.to_categorical(np.array(y), num_classes=2)

# 4. Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 5. Model LSTM
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(64, return_sequences=True, input_shape=(window_size, X.shape[2])),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(2, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# 6. Training
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

# 7. Evaluasi
loss, acc = model.evaluate(X_test, y_test)
print(f"\n🎯 Akurasi LSTM di data uji: {acc:.2f}")

# 8. Fungsi prediksi dengan input urutan
def predict_action_sequence(sequence_df):
    window = sequence_df[features].tail(window_size).values.reshape(1, window_size, len(features)).astype(np.float32) # Ensure prediction input is also float32
    pred = model.predict(window)
    class_idx = np.argmax(pred)
    return label_enc.inverse_transform([class_idx])[0]

# 9. Contoh prediksi
print("\n🔍 Prediksi aksi berdasarkan 10 detik terakhir:")
print(predict_action_sequence(df.tail(10)))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 10, 64)         │        17,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 2)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 30,402 (118.76 KB)

 Trainable params: 30,402 (118.76 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.5328 - loss: 0.6936 - val_accuracy: 0.5667 - val_loss: 0.6852
Epoch 2/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5653 - loss: 0.6853 - val_accuracy: 0.5667 - val_loss: 0.6881
Epoch 3/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5609 - loss: 0.6870 - val_accuracy: 0.5667 - val_loss: 0.6857
Epoch 4/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5703 - loss: 0.6850 - val_accuracy: 0.5667 - val_loss: 0.6850
Epoch 5/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5609 - loss: 0.6889 - val_accuracy: 0.5667 - val_loss: 0.6843
Epoch 6/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5683 - loss: 0.6827 - val_accuracy: 0.5625 - val_loss: 0.6849
Epoch 7/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5397 - loss: 0.6900 - val_accuracy: 0.5667 - val_loss: 0.6863
Epoch 8/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5616 - loss: 0.6907 - val_accuracy: 0.5667 - val_lo